In [2]:
from base_vit import ViT
import torch
from lora import LoRA_ViT
import numpy as np
import matplotlib.pyplot as plt
from utils.dataloader import _kneeDataloader
from tqdm import tqdm

In [3]:
model = ViT('B_16_imagenet1k')
model.load_state_dict(torch.load('B_16_imagenet1k.pth'))
model.eval()
criterion = torch.nn.CrossEntropyLoss()

In [4]:
weights = model.state_dict()
trainable_keys = [k for k, v in weights.items() if v.requires_grad]
trainable_weights = {k: v.clone() for k, v in weights.items() if k in trainable_keys}

In [5]:
grid_size = 10
delta = 0.1
trainloader = _kneeDataloader(bs=4)[0]
loss_grid = np.zeros((grid_size, grid_size))
for i in range(grid_size):
    for j in range(grid_size):
        for k, v in weights.items():
            weights[k] = weights[k] + delta * (i - grid_size // 2) * v + delta * (
                j - grid_size // 2) * torch.randn_like(v)
        model.load_state_dict(weights)
        running_loss = 0.0
        for i, data in tqdm(enumerate(trainloader, 0)):
            inputs, labels = data

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item()

        loss_grid[i, j] = running_loss


8it [01:34, 11.98s/it]

In [ ]:
plt.imshow(loss_grid, cmap='jet', interpolation='nearest')
plt.colorbar()
plt.show()